# Quora Question Pairs

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Import packages

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import keras
import nltk
import os

In [3]:
CLEAN_DATA_FOLDER = "clean_data"
MODEL_DATA_FOLDER = "models"
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nbthakur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nbthakur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load the model

In [4]:
LEARNING_RATE = 1e-3
EMBED_NAME = "glove"
EMBED_DIMS = 100
ATTN_HEADS = 4
ATTN_DIMS = 64
FF_DIMS = 32
MAX_LEN = 85
FF_DROPOUT = 0.1
from models.final_model import create_transformer_model
model = create_transformer_model(
    ATTN_HEADS, ATTN_DIMS, EMBED_NAME, EMBED_DIMS, MAX_LEN, FF_DIMS, FF_DROPOUT, LEARNING_RATE
)
model.load_weights(os.path.join(MODEL_DATA_FOLDER, f"final_model.keras"))

/Users/nbthakur/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 76 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Load the vocabulary

In [5]:
vocab = pd.read_csv(
    os.path.join(CLEAN_DATA_FOLDER, "vocab.tsv"),
    sep="\t",
    keep_default_na=False,
)
vocab_mapping = {row["token"]: row["id"] for _, row in vocab.iterrows()}

## User Input

In [6]:
TEXT1 = "Are stocks and security the same?"
TEXT2 = "Are stocks considered security?"

## Tokenize the sentences

In [7]:
text1 = word_tokenize(TEXT1.lower())
text2 = word_tokenize(TEXT2.lower())

## Remove stop words

In [8]:
text1 = [token for token in text1 if token not in stopwords.words("english")]
text2 = [token for token in text2 if token not in stopwords.words("english")]

## Convert words to token IDs

In [9]:
text1 = [vocab_mapping[token] for token in text1]
text2 = [vocab_mapping[token] for token in text2]

## Make predictions using the Model

In [10]:
text1 = keras.utils.pad_sequences([text1], padding="post")
text2 = keras.utils.pad_sequences([text2], padding="post")
y_pred = model.predict((text1, text2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step


## Interpret the results

In [11]:
if y_pred.round().astype(int).squeeze():
    print("The input texts are the same.")
else:
    print("The input texts are not the same.")

The input texts are the same.
